In [1]:
!nvidia-smi

Tue Nov 30 14:40:44 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:84:00.0 Off |                  N/A |
| 24%   35C    P8    11W / 250W |   4609MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:85:00.0 Off |                  N/A |
| 22%   

In [2]:
import torch
from torch import nn
import torch.nn.functional as F

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import os
import pickle
import numpy as np
import math
import matplotlib.pyplot as plt

In [3]:
class DeapS2SDatasetClassification(torch.utils.data.Dataset):
    
    def __init__(self, path):

        _, _, filenames = next(os.walk(path))
        filenames = sorted(filenames)
        all_data = []
        all_label = []
        for dat in filenames:
            temp = pickle.load(open(os.path.join(path, dat), 'rb'), encoding='latin1')
            all_data.append(temp['data'])
            all_label.append(temp['labels'][:,:1])

        self.data = np.vstack(all_data)
        self.label = np.vstack(all_label)
        del temp, all_data, all_label

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        single_data = self.data[idx]
        single_label = (self.label[idx] > 5).astype(float)
        
        batch = {
            'data': torch.Tensor(single_data),
            'label': torch.Tensor(single_label)
        }

        return batch

In [4]:
#@title Dataset Parameters { vertical-output: true }
model_type = "classification" #@param ["classification", "regression"]
batch_size = 8 #@param {type:"integer"}

if model_type == "classification":
    dataset = DeapS2SDatasetClassification('data_preprocessed_python')


torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
train_ind = int(0.8 * len(dataset))
train_set = torch.utils.data.Subset(dataset, indices[:train_ind])
val_set = torch.utils.data.Subset(dataset, indices[train_ind:])
del dataset

print(len(train_set))
print(len(val_set))

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, pin_memory=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=False, pin_memory=True)


1024
256


In [5]:
class ClassificationLSTM(nn.Module):
    def __init__(self, n_layers, in_features, emb_dim, out_features=1):
        super(ClassificationLSTM, self).__init__()

        self.lstm = nn.LSTM(in_features, emb_dim, n_layers, bidirectional=True,dropout=0.2)
        self.relu = nn.ReLU()
        self.out = nn.Linear(emb_dim * 2, out_features)
        self.dropout = nn.Dropout(0.2)
        self.sig = nn.Sigmoid()
    
    def forward(self, x):
        _, (hn, cn) = self.lstm(x)
        hidden = self.dropout(torch.cat((hn[-2,:,:], hn[-1,:,:]), dim = 1))
        return self.sig(self.out(self.relu(hidden)))
        

In [6]:
if model_type == "classification":
    model = ClassificationLSTM(2, 40, 256)
    loss_fn = nn.BCELoss()


model.cuda()

EPOCH = 15
lr = 0.0001
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

In [7]:
train_loss_list = []
val_loss_list = []
val_over_all = np.inf
for epoch in range(EPOCH):
    model.train()
    train_loss = 0

    for i, batch in enumerate(train_loader):
        data = batch['data'].permute(2, 0, 1).cuda()
        label = batch['label'].cuda()

        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, label)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss_list.append(train_loss/len(train_loader))

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(val_loader):

            data = batch['data'].permute(2, 0, 1).cuda()
            label = batch['label'].cuda()
            output = model(data)
            loss = loss_fn(output, label)
            val_loss += loss.item()

    val_loss_list.append(val_loss/len(val_loader))
    print('Epoch : {} train_loss : {} val_loss : {}'.format(epoch, train_loss/len(train_loader), val_loss/len(val_loader)))  

    if val_loss_list[-1] < val_over_all:
        val_over_all = val_loss_list[-1]
        ckpt = {
            'model_dict': model.state_dict(),
            'eval_loss': val_loss_list[-1]
        }

        torch.save(ckpt, 'base_lstm.pt')

Epoch : 0 train_loss : 0.6903091161511838 val_loss : 0.6891791336238384
Epoch : 1 train_loss : 0.6619729460217059 val_loss : 0.6867890786379576
Epoch : 2 train_loss : 0.6375969862565398 val_loss : 0.6920644044876099
Epoch : 3 train_loss : 0.5909550855867565 val_loss : 0.7181844767183065
Epoch : 4 train_loss : 0.5179760453756899 val_loss : 0.7373541621491313
Epoch : 5 train_loss : 0.41611088684294373 val_loss : 0.7909911423921585
Epoch : 6 train_loss : 0.3066539923893288 val_loss : 0.9303814098238945
Epoch : 7 train_loss : 0.24159941598190926 val_loss : 0.9409756232053041
Epoch : 8 train_loss : 0.17151343960722443 val_loss : 1.2017004219815135
Epoch : 9 train_loss : 0.12643137469422072 val_loss : 1.206684898585081
Epoch : 10 train_loss : 0.11794311651829048 val_loss : 1.2494973381981254
Epoch : 11 train_loss : 0.09003518785175402 val_loss : 1.4474006835371256
Epoch : 12 train_loss : 0.15321640261026914 val_loss : 1.392175069078803
Epoch : 13 train_loss : 0.10519062899402343 val_loss : 1

In [8]:
ckpt = torch.load('base_lstm.pt')
model.load_state_dict(ckpt['model_dict'])
model.cuda()
model.eval()

if model_type == "classification":

    

    fin_targets = []
    fin_outputs = []
    with torch.no_grad():
        for i, batch in enumerate(val_loader):

            data = batch['data'].permute(2, 0, 1).cuda()
            label = batch['label']
            output = model(data)
            fin_targets.append(label.numpy())
            fin_outputs.append(np.asarray((output.cpu().detach().numpy()>0.5), dtype=np.int))

    acc = accuracy_score(np.vstack(fin_outputs).flatten(), np.vstack(fin_targets).flatten())
    precision = precision_score(np.vstack(fin_outputs).flatten(), np.vstack(fin_targets).flatten())
    recall = recall_score(np.vstack(fin_outputs).flatten(), np.vstack(fin_targets).flatten())
    f1score = f1_score(np.vstack(fin_outputs).flatten(), np.vstack(fin_targets).flatten())

    print('Accuracy : {}'.format(acc))
    print('Precision: {}'.format(precision))
    print('Recall: {}'.format(recall))
    print('F1score: {}'.format(f1score))

/tmp/ipykernel_725/4120579341.py:19: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fin_outputs.append(np.asarray((output.cpu().detach().numpy()>0.5), dtype=np.int))


Accuracy : 0.5390625
Precision: 0.7785714285714286
Recall: 0.5561224489795918
F1score: 0.6488095238095238


In [9]:
print(model)

ClassificationLSTM(
  (lstm): LSTM(40, 256, num_layers=2, dropout=0.2, bidirectional=True)
  (relu): ReLU()
  (out): Linear(in_features=512, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (sig): Sigmoid()
)


In [10]:
dataiter = iter(train_loader)
data = dataiter.next()
images, labels = data['data'],data['label']
print(images.shape)
print(labels.shape)

torch.Size([8, 40, 8064])
torch.Size([8, 1])
